# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [111]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [112]:
# your code here
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [113]:
# your code here
# Sub Problem 1: How to aggregate the amount_spent for unique customers?
customer_total = orders.groupby('CustomerID').agg({'amount_spent':'sum'})

# Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
vip_threshold = customer_total['amount_spent'].quantile(0.95)
preferred_threshold = customer_total['amount_spent'].quantile(0.75)

vip_customers = customer_total[customer_total['amount_spent'] > vip_threshold]
preferred_customers = customer_total[(customer_total['amount_spent'] > preferred_threshold) & (customer_total['amount_spent'] <= vip_threshold)]

# Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?
customer_total['customer_label'] = 'Normal'
customer_total.loc[customer_total.index.isin(vip_customers.index), 'customer_label'] = 'VIP'
customer_total.loc[customer_total.index.isin(preferred_customers.index), 'customer_label'] = 'Preferred'

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [119]:
# Filter the DataFrame to include only VIP customers
vip_customers_df = orders[orders['CustomerID'].isin(vip_customers.index)]

# Count unique VIP customers by country
vip_customers_by_country = vip_customers_df.groupby('Country')['CustomerID'].nunique()

# Print VIP customer counts by country
print("VIP customers by country:")
print(vip_customers_by_country)

# A country with the mist VIPs:
print(f'A country with the most VIP customers is: {vip_customers_by_country.idxmax()} with {vip_customers_by_country.max()} customers')


VIP customers by country:
Country
Australia            2
Belgium              1
Channel Islands      1
Cyprus               1
Denmark              1
EIRE                 2
Finland              1
France               9
Germany             10
Japan                2
Netherlands          1
Norway               1
Portugal             2
Singapore            1
Spain                2
Sweden               1
Switzerland          3
United Kingdom     177
Name: CustomerID, dtype: int64
A country with the most VIP customers is: United Kingdom with 177 customers


## Q3: How to identify which country has the most VIP+Preferred Customers

*   List item
*   List item

combined?

In [120]:
# Filter the DataFrame to include only Preferred customers
preferred_customers_df = orders[orders['CustomerID'].isin(preferred_customers.index)]

# Count unique Preferred customers by country
preferred_customers_by_country = preferred_customers_df.groupby('Country')['CustomerID'].nunique()

# Print Preferred customer counts by country
print("Preferred customers by country:")
print(preferred_customers_by_country)

# Identify and print the country with the most Preferred customers
print(f'A country with the most preferred customers is: {preferred_customers_by_country.idxmax()} with {preferred_customers_by_country.max()} customers')


Preferred customers by country:
Country
Australia            2
Austria              3
Belgium             11
Canada               1
Channel Islands      3
Cyprus               3
Denmark              2
EIRE                 1
Finland              4
France              20
Germany             29
Greece               1
Iceland              1
Israel               2
Italy                5
Japan                2
Lebanon              1
Malta                1
Norway               6
Poland               1
Portugal             5
Spain                7
Sweden               1
Switzerland          6
United Kingdom     755
Name: CustomerID, dtype: int64
A country with the most preferred customers is: United Kingdom with 755 customers


In [121]:
#seeing that both VIP and preferred customers are mostly located in United Kingdom we can jsut sun up the values
most_vip_and_preferred_customers = vip_customers_by_country.max() + preferred_customers_by_country.max()
print(f'The total number of most VIP and preferred customers is: {most_vip_and_preferred_customers}')

The total number of most VIP and preferred customers is: 932


In [122]:
# Extract counts for the United Kingdom
uk_vip_count = vip_customers_by_country.get('United Kingdom', 0)
uk_preferred_count = preferred_customers_by_country.get('United Kingdom', 0)

# Calculate the total for the United Kingdom
total_uk_customers = uk_vip_count + uk_preferred_count

print(f'The total number of VIP and Preferred customers in the United Kingdom is: {total_uk_customers}')


The total number of VIP and Preferred customers in the United Kingdom is: 932
